In [1]:
# modules
import sys
sys.path.insert(0, '../')

import numpy as np
from numpy.core.fromnumeric import mean
import pandas as pd
from pathlib import Path
import os
import matplotlib.pyplot as plt
#from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.metrics import f1_score, accuracy_score
import argparse

In [2]:
%cd /home/hirano/work/Competition/Nishika_cable/src/
%load_ext autoreload
%autoreload 2

/home/hirano/work/Competition/Nishika_cable/src


In [3]:
import argparse
parser = argparse.ArgumentParser(description='Nishik_cable')
parser.add_argument('--version', default='006')
parser.add_argument('--gpu', default='0', type=int)
args = parser.parse_args(args=[])

In [4]:
version = args.version
folds = [0,1,2,3,4]

sub = pd.read_csv('../input/test_with_fold.csv')
pred = np.zeros(len(sub))

train = pd.read_csv('../input/train_with_fold.csv')
oof = pd.DataFrame()

In [5]:
for fold in folds:
    print(fold)
    df = pd.read_csv(f'../output/{version}/{fold}/sub_{version}.csv')
    if df.target.isnull().sum():
        print('null detected!!')
        df.target = df.target.fillna(0)
    pred = df.target.values
    sub['fold_{}'.format(fold)] = np.array(pred, dtype=np.uint8)
    
    oof_name = [x for x in os.listdir(f'../output/{version}/{fold}/') if 'oof' in x]
    _oof = pd.read_csv(f'../output/{version}/{fold}/{oof_name[0]}')
    oof = pd.concat([oof, _oof], axis=0)

0
1
2
3
4


In [9]:
df

,target
0,8
1,4
2,8
3,4
4,1
...,...
2195,4
2196,1
2197,7
2198,6


In [6]:
assert len(oof)==len(train)
#score = np.sqrt(mean_squared_error(oof.target.values, oof.pred.values))
score = f1_score(oof.target.values, oof.pred.values, average='micro')
print(score)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [7]:
sub

,filename,class,fold,target,fold_0,fold_1,fold_2,fold_3,fold_4
0,997195ba.jpg,USB_Type_C,0,0,8,8,8,8,8
1,9f3f263d.jpg,USB_Type_C,0,0,4,4,4,4,4
2,1072aa7d.jpg,USB_Type_C,0,0,8,8,8,8,8
3,f8047293.jpg,USB_Type_C,0,0,4,4,4,4,4
4,3dccb562.jpg,USB_Type_C,0,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
2195,4eae2d46.jpg,USB_Type_C,0,0,4,4,4,4,4
2196,ceadcae4.jpg,USB_Type_C,0,0,1,1,1,1,1
2197,4a5a3544.jpg,USB_Type_C,0,0,7,7,7,7,7
2198,bbc3c7b6.jpg,USB_Type_C,0,0,6,6,6,6,6


In [109]:
results_concat = np.array([sub['fold_0'], sub['fold_1'], sub['fold_2'], sub['fold_3'], sub['fold_4']])

In [110]:
results = [*map(lambda x: np.argmax(np.bincount(x)), results_concat.T)]

In [124]:
label_dict = {
   0 : 'HDMI',
   1 : 'USB_Mini',
   2 : 'Dock',
   3 : 'Lightning',
   4 : 'USB_Micro_B',
   5 : 'Mini_DisplayPort',
   6 : 'RJ_45',
   7 : 'USB_Type_A',
   8 : 'USB_Type_C',
   9 : 'USB_Micro_B_3.1',
  10 : 'USB_Type_B',
  11 : 'Lightning_T',
  12 : 'USB_Micro_B_W',
  13 : 'VGA',
  14 : 'DisplayPort'
}

In [126]:
targets = []
for i in range(len(results)):
    targets.append(label_dict[results[i]])
print(len(targets))

2200


In [135]:
sub['class'] = targets

In [136]:
sub

,filename,class,fold,target,fold_0,fold_1,fold_2,fold_3,fold_4
0,997195ba.jpg,USB_Type_C,0,USB_Type_C,8,8,8,8,8
1,9f3f263d.jpg,USB_Micro_B,0,USB_Micro_B,4,4,4,4,4
2,1072aa7d.jpg,USB_Type_C,0,USB_Type_C,8,8,8,8,8
3,f8047293.jpg,USB_Micro_B,0,USB_Micro_B,4,4,4,4,4
4,3dccb562.jpg,USB_Mini,0,USB_Mini,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
2195,4eae2d46.jpg,USB_Micro_B,0,USB_Micro_B,4,4,4,4,4
2196,ceadcae4.jpg,USB_Mini,0,USB_Mini,1,1,1,1,1
2197,4a5a3544.jpg,USB_Type_A,0,USB_Type_A,7,7,7,7,7
2198,bbc3c7b6.jpg,RJ_45,0,RJ_45,6,6,6,6,6


In [137]:
sub[['filename', 'class']].to_csv(f'../output/{version}/submission_{version}.csv', index=False)

In [82]:
nrow = 100
ncol = 400_000
np.random.seed(0)
df = pd.DataFrame(np.random.choice([-1,0,1],(nrow,ncol)))

In [83]:
df

,0,1,2,3,4,5,6,7,8,9,...,399990,399991,399992,399993,399994,399995,399996,399997,399998,399999
0,-1,0,-1,0,0,1,-1,1,-1,-1,...,1,0,1,-1,1,0,1,-1,0,0
1,-1,1,1,-1,1,1,1,-1,-1,0,...,-1,-1,0,-1,1,1,-1,1,0,0
2,0,0,1,1,1,0,-1,1,-1,1,...,-1,0,1,-1,-1,0,-1,1,0,1
3,1,1,0,1,0,-1,0,0,-1,1,...,1,1,-1,0,-1,0,1,0,0,0
4,-1,0,1,-1,0,1,0,-1,1,-1,...,0,-1,-1,0,-1,0,1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,-1,0,0,-1,-1,0,0,-1,1,...,0,1,0,-1,0,-1,1,-1,0,0
96,0,-1,0,1,1,-1,1,0,-1,-1,...,1,0,0,1,1,0,0,1,1,1
97,1,-1,-1,1,0,1,1,0,1,0,...,-1,1,0,0,1,0,0,-1,-1,0
98,-1,-1,-1,1,1,1,1,0,1,1,...,1,1,-1,-1,0,-1,-1,-1,1,-1


In [84]:
arr = df.to_numpy()

In [61]:
max, min, ncol = arr.max(), arr.min(), arr.shape[1]

In [65]:
offset = np.arange(ncol) * (max-min+1) - min

In [67]:
np.bincount((arr+offset).ravel()).reshape(ncol, -1).argmax(1) + min

array([ 0, -1, -1, ...,  0,  0,  1])

In [85]:
arr.shape

(100, 400000)